In [3]:
import os
import time
import pandas as pd
import numpy as np
import openpyxl
from tqdm import tqdm
from dotenv import load_dotenv
import warnings

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from sklearn.metrics.pairwise import cosine_similarity

# Suppress LangChain tracer warnings
warnings.filterwarnings("ignore", message="Error in LangChainTracer")

# --- Configuration ---
load_dotenv()

# Disable LangChain tracing to avoid serialization errors
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_TRACING"] = "false"

# Expected input file structure:
# - data/df_with_embeddings.parquet with columns:
#   - 'combined_text': the text content
#   - 'embedding': OpenAI text-embedding-3-small vectors

config = {
    "embedding_model": "text-embedding-3-small",
    "llm_model": "gpt-4o-mini",  # Changed to gpt-4o-mini for cost efficiency
    "retrieval_k": 5,
    "categories_1": ["IRB", "SA"],  # Updated with real categories
    "categories_2": ["PD", "LGD", "In-default LGD"],  # Updated with real categories
    "cost_embedding_per_1m_tokens": 0.02,
    "cost_llm_input_per_1m_tokens": 0.15,  # Updated for gpt-4o-mini pricing
    "cost_llm_output_per_1m_tokens": 0.60,  # Updated for gpt-4o-mini pricing
    "use_cache": True,
}

# --- Helper Functions & Setup ---

def create_dummy_golden_file():
    """Creates dummy golden source file for demonstration if it doesn't exist."""
    golden_file = "golden_source.xlsx"
    if not os.path.exists(golden_file):
        print(f"Creating dummy file: {golden_file}")
        # Adjusted TOC numbers to match the format we'll generate (row_index//10+1).(row_index%10+1)
        pd.DataFrame({
            "Question_ID": [f"Q{i}" for i in range(1, 6)],
            "Question_Text": [
                "What are the IRB requirements for PD models?",
                "How is LGD calculated under the SA approach?",
                "What is the difference between PD and in-default LGD?",
                "What are the key components of IRB models?",
                "How does SA handle LGD estimation?"
            ],
            "TOC_1": ["1.1", "1.2", "1.3", "1.4", "1.5"],
            "TOC_2": ["2.1", "2.2", "2.3", "2.4", "2.5"],
            "TOC_3": ["3.1", "3.2", "3.3", "3.4", "3.5"],
            "TOC_4": ["4.1", "4.2", "4.3", "4.4", "4.5"],
            "TOC_5": ["5.1", "5.2", "5.3", "5.4", "5.5"],
        }).to_excel(golden_file, index=False)
        print(f"Note: Please update {golden_file} with actual TOC references from your documents.")

# --- System Components ---

class CostTracker:
    def __init__(self, config):
        self.config, self.total_cost, self.cost_breakdown = config, 0, {
            "setup_categorization": 0, "setup_embedding": 0, "query_categorization": 0,
            "query_embedding": 0, "query_llm_context": 0
        }
    def _calculate_cost(self, tokens, type):
        if type == "embedding": return (tokens / 1_000_000) * self.config['cost_embedding_per_1m_tokens']
        if type == "llm_input": return (tokens / 1_000_000) * self.config['cost_llm_input_per_1m_tokens']
        if type == "llm_output": return (tokens / 1_000_000) * self.config['cost_llm_output_per_1m_tokens']
        return 0
        
    def add_cost(self, tokens, type, component):
        cost = self._calculate_cost(tokens, type)
        self.total_cost += cost
        if component in self.cost_breakdown: self.cost_breakdown[component] += cost
        return cost
        
    def get_summary(self): return {"total_cost": self.total_cost, "breakdown": self.cost_breakdown}

class DocumentCategorizer:
    def __init__(self, category_1_list, category_2_list, llm, cost_tracker):
        self.llm, self.cost_tracker = llm, cost_tracker
        class Categories(BaseModel):
            category_1: str = Field(description=f"The approach/framework from the list: {category_1_list}")
            category_2: str = Field(description=f"The risk parameter from the list: {category_2_list}")
        self.parser = PydanticOutputParser(pydantic_object=Categories)
        self.prompt = PromptTemplate(
            template="Analyze the text and assign categories.\n{format_instructions}\nText: \"{text}\"",
            input_variables=["text"], partial_variables={"format_instructions": self.parser.get_format_instructions()},
        )
        self.chain = self.prompt | self.llm | self.parser
        
    def categorize_text(self, text: str):
        try:
            # Ensure text is a string
            text = str(text) if not isinstance(text, str) else text
            # Limit text length to avoid token limits
            text = text[:2000] if len(text) > 2000 else text
            
            input_tokens, result = len(text) // 4, self.chain.invoke({"text": text})
            output_tokens = len(str(result)) // 4
            self.cost_tracker.add_cost(input_tokens, "llm_input", "setup_categorization")
            self.cost_tracker.add_cost(output_tokens, "llm_output", "setup_categorization")
            return result.category_1, result.category_2
        except Exception as e: 
            print(f"Error categorizing text: {e}")
            return "Uncategorized", "Uncategorized"
            
    def categorize_dataset(self, df: pd.DataFrame):
        # Make a copy to avoid modifying the original
        df_copy = df.copy()
        categories = []
        
        for idx, row in tqdm(df_copy.iterrows(), total=len(df_copy), desc="Categorizing Documents"):
            # Extract text as string
            text = str(row['Text']) if pd.notna(row['Text']) else ""
            cat1, cat2 = self.categorize_text(text)
            categories.append((cat1, cat2))
        
        df_copy['Category_1'] = [cat[0] for cat in categories]
        df_copy['Category_2'] = [cat[1] for cat in categories]
        return df_copy

class DocumentRetriever:
    def __init__(self, df, config, cost_tracker, skip_embedding_generation=False):
        self.df, self.config, self.cost_tracker, self.llm_cache = df.copy(), config, cost_tracker, {}
        self.embedding_model = OpenAIEmbeddings(model=config["embedding_model"])
        self.llm = ChatOpenAI(model=config["llm_model"], temperature=0)
        self.categorizer = DocumentCategorizer(config["categories_1"], config["categories_2"], self.llm, cost_tracker)
        
        if not skip_embedding_generation:
            print("Generating document embeddings...")
            texts, total_tokens = self.df['Text'].tolist(), sum(len(t)//4 for t in self.df['Text'])
            self.cost_tracker.add_cost(total_tokens, 'embedding', 'setup_embedding')
            self.df['embedding'] = self.embedding_model.embed_documents(texts)
        else:
            print("Using existing embeddings from dataframe...")
            # Ensure embeddings are in the right format (list of floats)
            if 'embedding' in self.df.columns:
                self.df['embedding'] = self.df['embedding'].apply(lambda x: x if isinstance(x, list) else list(x))
        
    def _categorize_question(self, question, question_id):
        if hasattr(self, '_cat_cache') and question_id in self._cat_cache: return self._cat_cache[question_id]
        input_tokens, (cat1, cat2) = len(question)//4, self.categorizer.categorize_text(question)
        output_tokens = len(cat1)//4 + len(cat2)//4
        self.cost_tracker.add_cost(input_tokens, 'llm_input', 'query_categorization')
        self.cost_tracker.add_cost(output_tokens, 'llm_output', 'query_categorization')
        if not hasattr(self, '_cat_cache'): self._cat_cache = {}
        self._cat_cache[question_id] = (cat1, cat2)
        return cat1, cat2
        
    def retrieve(self, s_name, question, k, q_id):
        start = time.time()
        if s_name == 'A': tocs, cost = self.retrieve_by_embedding(question, k)
        elif s_name == 'B': tocs, cost = self.retrieve_by_category(question, k, q_id)
        elif s_name == 'C': tocs, cost = self.retrieve_hybrid(question, k, q_id)
        elif s_name == 'D': tocs, cost = self.retrieve_full_context(question, k)
        else: raise ValueError(f"Unknown strategy: {s_name}")
        return tocs, (time.time() - start) * 1000, cost
        
    def retrieve_by_embedding(self, question, k):
        q_emb = self.embedding_model.embed_query(question)
        cost = self.cost_tracker.add_cost(len(question)//4, "embedding", "query_embedding")
        sims = cosine_similarity([q_emb], np.array(self.df['embedding'].tolist()))[0]
        return self.df.iloc[np.argsort(sims)[::-1][:k]]['TOC_Number'].tolist(), cost
        
    def retrieve_by_category(self, question, k, q_id):
        cat1, cat2 = self._categorize_question(question, q_id)
        df = self.df[(self.df['Category_1'] == cat1) & (self.df['Category_2'] == cat2)]
        if df.empty: df = self.df[self.df['Category_1'] == cat1]
        return df['TOC_Number'].head(k).tolist(), 0
        
    def retrieve_hybrid(self, question, k, q_id):
        cost = self.cost_tracker.add_cost(len(question)//4, "embedding", "query_embedding")
        cat1, cat2 = self._categorize_question(question, q_id)
        # --- THIS IS THE CORRECTED LINE ---
        df = pd.concat([
            self.df[(self.df['Category_1']==cat1)&(self.df['Category_2']==cat2)], 
            self.df[self.df['Category_1']==cat1]
        ]).drop_duplicates(subset=['TOC_Number']).reset_index(drop=True)
        # --- END OF CORRECTION ---
        if df.empty: return [], cost
        q_emb = self.embedding_model.embed_query(question)
        df['sim'] = cosine_similarity([q_emb], np.array(df['embedding'].tolist()))[0]
        return df.sort_values('sim', ascending=False).head(k)['TOC_Number'].tolist(), cost
        
    def retrieve_full_context(self, question, k):
        if self.config['use_cache'] and question in self.llm_cache: return self.llm_cache[question][0], 0
        context_str = "\n".join([f"TOC {row['TOC_Number']}: {row['Text']}" for _, row in self.df.iterrows()])
        prompt = f"""Given the document context below, identify the TOP {k} `TOC_Number`s most relevant to the user's question. Return only a comma-separated list of TOC numbers (e.g., 1.1, 2.3, 3.2).
CONTEXT:
---
{context_str}
---
QUESTION: "{question}"
Relevant TOC_Numbers:
"""
        res = self.llm.invoke(prompt).content.strip()
        tocs = [t.strip() for t in res.split(',')]
        cost = self.cost_tracker.add_cost(len(prompt)//4, 'llm_input', 'query_llm_context') + \
               self.cost_tracker.add_cost(len(res)//4, 'llm_output', 'query_llm_context')
        if self.config['use_cache']: self.llm_cache[question] = (tocs, cost)
        return tocs, cost

class Evaluator:
    def __init__(self, golden_source_df, k):
        self.k = k
        self.golden_df = golden_source_df
        toc_cols = [f'TOC_{i}' for i in range(1, 6)]
        self.golden_map = {row['Question_ID']: set(row[toc_cols].astype(str).values) for _, row in self.golden_df.iterrows()}
            
    def evaluate_run(self, retrieved_tocs, question_id):
        golden_tocs = self.golden_map.get(question_id, set())
        matches = len(set(retrieved_tocs).intersection(golden_tocs))
        precision = matches / self.k if self.k > 0 else 0
        return {'matches': matches, 'precision': precision, 'retrieved_tocs': ", ".join(map(str, retrieved_tocs)), 'golden_tocs': ", ".join(map(str, sorted(list(golden_tocs))))}

    def generate_report(self, all_results, cost_tracker, categorized_df):
        print("\nGenerating final report...")
        report_filename = "retrieval_evaluation_report.xlsx"
        strategy_map = {'A': 'A: Embedding', 'B': 'B: Categories', 'C': 'C: Hybrid', 'D': 'D: Full Context'}

        # 1. Per-Question Accuracy Summary
        per_q_summary_data = []
        for q_id, q_results in all_results.items():
            golden_tocs_set = self.golden_map.get(q_id, set())
            golden_tocs_str = ", ".join(map(str, sorted(list(golden_tocs_set))))
            for s_code, result in q_results.items():
                if s_code == 'categorization_cost': continue
                eval_metrics = self.evaluate_run(result['tocs'], q_id)
                per_q_summary_data.append({
                    'Question_ID': q_id,
                    'Strategy': strategy_map[s_code],
                    'Golden TOCs': golden_tocs_str,
                    'Retrieved TOCs': eval_metrics['retrieved_tocs'],
                    'Overlap %': f"{eval_metrics['precision']:.0%}"
                })
        per_q_summary_df = pd.DataFrame(per_q_summary_data)

        # 2. Detailed Per-Question Metrics
        detailed_metrics_data = []
        for q_id, q_results in all_results.items():
            row = {'Question_ID': q_id, 'Question_Text': self.golden_df[self.golden_df['Question_ID'] == q_id]['Question_Text'].iloc[0]}
            q_cat_cost = q_results.get('categorization_cost', 0)
            for s_code, result in q_results.items():
                if s_code == 'categorization_cost': continue
                eval_metrics = self.evaluate_run(result['tocs'], q_id)
                final_cost = result['cost'] + (q_cat_cost if s_code in ['B', 'C'] else 0)
                row[f'{s_code}_Matches'] = eval_metrics['matches']
                row[f'{s_code}_Precision'] = eval_metrics['precision']
                row[f'{s_code}_Latency(ms)'] = result['latency']
                row[f'{s_code}_Cost($)'] = final_cost
            detailed_metrics_data.append(row)
        detailed_metrics_df = pd.DataFrame(detailed_metrics_data)

        # 3. Strategy-Level Summary
        summary_data = []
        for s_code, s_name in strategy_map.items():
            summary_data.append({
                'Strategy': s_name,
                'Avg Precision': f"{detailed_metrics_df[f'{s_code}_Precision'].mean():.2%}",
                'Total Query Cost ($)': f"{detailed_metrics_df[f'{s_code}_Cost($)'].sum():.6f}",
                'Avg Latency (ms)': f"{detailed_metrics_df[f'{s_code}_Latency(ms)'].mean():.2f}"
            })
        summary_df = pd.DataFrame(summary_data)
        
        # 4. Cost Breakdown
        costs = cost_tracker.get_summary()
        cost_df = pd.DataFrame({
            'Component': [
                'Setup: Document Categorization', 'Setup: Document Embeddings', '---',
                'Total Query Costs (Aggregated)', '---', 'Total Estimated Cost (Setup + Query)'
            ],
            'Cost ($)': [
                f"{costs['breakdown']['setup_categorization']:.6f}", f"{costs['breakdown']['setup_embedding']:.6f}", '---',
                f"{costs['total_cost'] - costs['breakdown']['setup_categorization'] - costs['breakdown']['setup_embedding']:.6f}", '---',
                f"{costs['total_cost']:.6f}"
            ]
        })

        # 5. Write all DataFrames to a multi-sheet Excel file
        with pd.ExcelWriter(report_filename, engine='openpyxl') as writer:
            per_q_summary_df.to_excel(writer, sheet_name='Per-Question Accuracy Summary', index=False)
            summary_df.to_excel(writer, sheet_name='Strategy-Level Summary', index=False)
            detailed_metrics_df.to_excel(writer, sheet_name='Detailed Per-Question Metrics', index=False)
            cost_df.to_excel(writer, sheet_name='Cost Breakdown', index=False)
            categorized_df.drop(columns=['embedding'], errors='ignore').to_excel(
                writer, sheet_name='Categorized Documents', index=False
            )
            
        print("\n--- Evaluation Report ---")
        print("Per-Question Accuracy Summary (Top 5 rows):")
        print(per_q_summary_df.head().to_string(index=False))
        print("\nStrategy-Level Summary:")
        print(summary_df.to_string(index=False))
        print(f"\nFull report with 5 sheets saved to '{report_filename}'")
        print(f"Total estimated cost for this run: ${costs['total_cost']:.4f}")

# --- Jupyter Notebook Execution Code ---
# Create dummy golden file if needed
create_dummy_golden_file()

# Check for OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY environment variable not set. Please create a .env file or set it in your notebook.")

# Initialize components
cost_tracker = CostTracker(config)
llm = ChatOpenAI(model=config["llm_model"], temperature=0)

# Read the real parquet file with embeddings
print("Loading document dataset from parquet file...")
docs_df = pd.read_parquet("data/df_with_embeddings.parquet")
docs_df = docs_df.head(20)

# Print available columns for debugging
print(f"Available columns in parquet file: {docs_df.columns.tolist()}")
print(f"Number of documents: {len(docs_df)}")

# Check for required columns
if 'combined_text' not in docs_df.columns:
    raise ValueError("No 'combined_text' column found in the parquet file. Please check column names.")
if 'embedding' not in docs_df.columns:
    raise ValueError("No 'embedding' column found in the parquet file.")

# Keep only relevant columns to save memory and avoid duplicates
docs_df = docs_df[['combined_text', 'embedding']].copy()
docs_df = docs_df.rename(columns={'combined_text': 'Text'})

# Create TOC_Number based on index
docs_df['TOC_Number'] = [f"{i//10+1}.{i%10+1}" for i in range(len(docs_df))]

# Verify embedding format (convert to list if needed)
first_embedding = docs_df['embedding'].iloc[0]
if isinstance(first_embedding, np.ndarray):
    print("Converting embeddings from numpy arrays to lists...")
    docs_df['embedding'] = docs_df['embedding'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

print(f"Sample text (first 200 chars): {docs_df['Text'].iloc[0][:200]}...")
print(f"Embedding dimension: {len(docs_df['embedding'].iloc[0])}")

# Load golden source
golden_df = pd.read_excel("golden_source.xlsx")

# Categorize documents
print("\nCategorizing documents...")
categorizer = DocumentCategorizer(config["categories_1"], config["categories_2"], llm, cost_tracker)
categorized_df = categorizer.categorize_dataset(docs_df)
categorized_df.to_parquet("document_dataset_categorized.parquet", index=False)
print("Categorized documents saved to parquet.")

# Initialize retriever with existing embeddings
skip_embedding_generation = 'embedding' in categorized_df.columns
retriever = DocumentRetriever(categorized_df, config, cost_tracker, skip_embedding_generation=skip_embedding_generation)

# Run retrieval strategies for all questions
all_results = {}
print("\nRunning retrieval strategies for all questions...")
for _, row in tqdm(golden_df.iterrows(), total=len(golden_df), desc="Evaluating Questions"):
    q_id, question = row['Question_ID'], row['Question_Text']
    cost_tracker.cost_breakdown['query_categorization'] = 0
    
    all_results[q_id] = {
        s_code: {'tocs': t, 'latency': l, 'cost': c}
        for s_code, (t, l, c) in zip(
            ['A', 'B', 'C', 'D'],
            [retriever.retrieve(s, question, config["retrieval_k"], q_id) for s in ['A', 'B', 'C', 'D']]
        )
    }
    all_results[q_id]['categorization_cost'] = cost_tracker.cost_breakdown['query_categorization']
    if hasattr(retriever, '_cat_cache'): retriever._cat_cache.clear()

# Generate evaluation report
evaluator = Evaluator(golden_df, config["retrieval_k"])
evaluator.generate_report(all_results, cost_tracker, retriever.df)

Loading document dataset from parquet file...
Available columns in parquet file: ['Part', 'Title', 'Chapter', 'Section', 'Subsection', 'Part_Heading', 'Title_Heading', 'Chapter_Heading', 'Section_Heading', 'Subsection_Heading', 'Token_Count', 'Ends_With_Dot', 'Article_Number', 'Article_Heading', 'Text', 'Text_With_Pagebreaks', 'combined_text', 'embedding']
Number of documents: 20
Converting embeddings from numpy arrays to lists...
Sample text (first 200 chars): Article_Heading: Article 1 - Scope...
Embedding dimension: 1536

Categorizing documents...


Categorizing Documents: 100%|██████████████████████████████████████████████████████████| 20/20 [00:17<00:00,  1.17it/s]


Categorized documents saved to parquet.
Using existing embeddings from dataframe...

Running retrieval strategies for all questions...


Evaluating Questions: 100%|██████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.31s/it]


Generating final report...

--- Evaluation Report ---
Per-Question Accuracy Summary (Top 5 rows):
Question_ID        Strategy             Golden TOCs           Retrieved TOCs Overlap %
         Q1    A: Embedding 1.1, 1.2, 2.2, 2.3, 4.1  1.8, 2.1, 1.2, 2.4, 2.6       20%
         Q1   B: Categories 1.1, 1.2, 2.2, 2.3, 4.1  1.1, 1.2, 1.3, 1.4, 1.5       40%
         Q1       C: Hybrid 1.1, 1.2, 2.2, 2.3, 4.1  1.8, 2.1, 1.2, 2.4, 2.6       20%
         Q1 D: Full Context 1.1, 1.2, 2.2, 2.3, 4.1  1.6, 1.5, 1.4, 1.3, 1.2       20%
         Q2    A: Embedding 1.1, 3.1, 3.2, 3.3, 4.1 2.5, 2.1, 1.10, 1.6, 2.8        0%

Strategy-Level Summary:
       Strategy Avg Precision Total Query Cost ($) Avg Latency (ms)
   A: Embedding        24.00%             0.000001           356.85
  B: Categories        28.00%             0.000010           674.10
      C: Hybrid        16.00%             0.000011           319.35
D: Full Context        24.00%             0.007716           960.11

Full report w

In [4]:
all_results

{'Q1': {'A': {'tocs': ['1.8', '2.1', '1.2', '2.4', '2.6'],
   'latency': 563.8754367828369,
   'cost': 2.2e-07},
  'B': {'tocs': ['1.1', '1.2', '1.3', '1.4', '1.5'],
   'latency': 817.7018165588379,
   'cost': 0},
  'C': {'tocs': ['1.8', '2.1', '1.2', '2.4', '2.6'],
   'latency': 540.198802947998,
   'cost': 2.2e-07},
  'D': {'tocs': ['1.6', '1.5', '1.4', '1.3', '1.2'],
   'latency': 803.7383556365967,
   'cost': 0.00154335},
  'categorization_cost': 1.6499999999999999e-06},
 'Q2': {'A': {'tocs': ['2.5', '2.1', '1.10', '1.6', '2.8'],
   'latency': 441.8022632598877,
   'cost': 2.2e-07},
  'B': {'tocs': [], 'latency': 648.1449604034424, 'cost': 0},
  'C': {'tocs': [], 'latency': 1.9271373748779297, 'cost': 2.2e-07},
  'D': {'tocs': ['1.6', '1.5', '1.4', '1.3', '1.2'],
   'latency': 705.0299644470215,
   'cost': 0.00154335},
  'categorization_cost': 1.6499999999999999e-06},
 'Q3': {'A': {'tocs': ['1.6', '1.2', '2.5', '1.4', '1.9'],
   'latency': 257.70044326782227,
   'cost': 2.6e-07},
 